In [3]:
source("helper.R")

max_expid<-get_max("Log")+1
exp_name<-"sensitivityOfStartPoint"
log_dir<-glue("Log/{max_expid}.{exp_name}")
logger=get_logger(log_dir, log_name = "log", debug = TRUE)
futile.logger::flog.info("start experiment", name = "log")

## get logger


seed_random<-123
n=30


set.seed(seed_random)

futile.logger::flog.info(glue("use seed: {seed_random}"), name = "log")

futile.logger::flog.info(glue("number of experiments: {n}"), name = "log")

data_ori <- as.data.frame(read_csv("data/weighted_priceIF2404_0317_417.csv"))



logpt<-log(data_ori$close)
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- as.POSIXct(as.character(data_ori$raw.timeindex[1:(length(data_ori$raw.timeindex)-1)]), format = "%Y-%m-%d %H:%M:%S")


date_timeindex<-as.numeric(date) 
date_timeindex <- (date_timeindex - min(date_timeindex))/86400 ## 1 day = 86400 seconds


dataChina = data.frame(logreturn = logreturn, times = date_timeindex)
data <- rbind(dataChina, data.frame(logreturn = NA, times = max(date_timeindex):(max(date_timeindex)+31)))
mesh <- fm_mesh_1d(dataChina$times)





INFO [2025-08-27 21:10:41] start experiment
INFO [2025-08-27 21:10:41] use seed: 123
INFO [2025-08-27 21:10:41] number of experiments: 30


Rows: 4664 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (1): close
dttm (1): raw.timeindex

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:


params <- data.frame(
  rspde.order = sample(2:6, n, replace = TRUE),
  prior.range.nominal = runif(n, 5, 300),
  prior.std.dev.nominal = runif(n, 0.3, 1.7)
)




# 

for(model_family in c("stochvol.t")){
  results_list <- list()
  mlik_vec <- numeric(n)


  futile.logger::flog.info(glue("Start fitting models:{model_family}"), name = "log")
  for (i in 1:n) {
    futile.logger::flog.info(glue("🔁 Fitting model {i} of {n} ..."), name = "log")
    
    # 提取参数
    rspde.order <- params$rspde.order[i]
    prior.range.nominal <- params$prior.range.nominal[i]
    prior.std.dev.nominal <- params$prior.std.dev.nominal[i]
    
    # 构建模型
    OU_model <- rspde.matern(
      mesh = mesh,
      parameterization = "matern",
      rspde.order = rspde.order,
      prior.range.nominal = prior.range.nominal,
      prior.std.dev.nominal = prior.std.dev.nominal
    )
    
    formula_OU <- logreturn ~ -1 + Intercept(1)+ field(times, model = OU_model)
    
    fit <- tryCatch(
      bru(
        formula_OU,
        family = model_family,
        data = data
      ),
      error = function(e) {
        futile.logger::flog.warn(glue("⚠️ Fit failed at i = {i} : {e$message}"), name = "log")
        return(NULL)
      }
    )
    
    # 如果成功拟合
    if (!is.null(fit)) {
      result_fit <- rspde.result(fit, "field", OU_model, parameterization = "matern")
      mlik_vec[i] <- fit$mlik[2]  # marginal log-likelihood
      results_list[[i]] <- list(
        param = params[i, ],
        mlik = fit$mlik[2],
        result = result_fit
      )
    } else {
      mlik_vec[i] <- NA
      results_list[[i]] <- list(param = params[i, ], mlik = NA, result = NULL)
    }
  }
  sorted_idx <- order(mlik_vec, decreasing = TRUE)


  results_list_sorted <- results_list[sorted_idx]


  table_out <- data.frame(
    param = character(),
    std.dev = numeric(),
    range = numeric(),
    nu = numeric(),
    likelihood = numeric(),
    stringsAsFactors = FALSE
  )
  results_list_sorted <- results_list[sorted_idx]
  for (i in seq_along(results_list_sorted)) {
    res <- results_list_sorted[[i]]
    
    if (is.null(res$result)) next 
    
    param_str <- with(res$param, glue::glue("({rspde.order}/{round(prior.range.nominal, 4)}/{round(prior.std.dev.nominal, 4)})"))
    
    summary_res <- summary(res$result)
    
    table_out[i, ] <- list(
      param = param_str,
      std.dev = round(summary_res$mean[1], 3),
      range   = round(summary_res$mean[2], 3),
      nu      = round(summary_res$mean[3], 3),
      likelihood = round(res$mlik, 3)
    )
  }

  table_out <- na.omit(table_out)


  table_string<-knitr::kable(
    table_out,
    format = "latex",
    booktabs = TRUE,
    linesep = ""
  )

  file_name <- glue("{log_dir}/model_with_{model_family}.tex")
  writeLines(table_string,  file_name)

}